In [4]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from transformers import pipeline
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import ollama
import json


In [2]:
df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/data/amazon-customer-reviews/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

# EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28332 entries, 0 to 28331
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   28332 non-null  object 
 1   dateAdded            28332 non-null  object 
 2   dateUpdated          28332 non-null  object 
 3   name                 28332 non-null  object 
 4   asins                28332 non-null  object 
 5   brand                28332 non-null  object 
 6   categories           28332 non-null  object 
 7   primaryCategories    28332 non-null  object 
 8   imageURLs            28332 non-null  object 
 9   keys                 28332 non-null  object 
 10  manufacturer         28332 non-null  object 
 11  manufacturerNumber   28332 non-null  object 
 12  reviews.date         28332 non-null  object 
 13  reviews.dateSeen     28332 non-null  object 
 14  reviews.didPurchase  9 non-null      object 
 15  reviews.doRecommend  16086 non-null 

In [4]:
df.shape

(28332, 24)

In [47]:
df.columns = df.columns.str.lower()

In [6]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,reviews.didpurchase,reviews.dorecommend,reviews.id,reviews.numhelpful,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht..."
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht..."
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht..."
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht..."


In [48]:
df.isnull().sum()

id                         0
dateadded                  0
dateupdated                0
name                       0
asins                      0
brand                      0
categories                 0
primarycategories          0
imageurls                  0
keys                       0
manufacturer               0
manufacturernumber         0
reviews.date               0
reviews.dateseen           0
reviews.didpurchase    28323
reviews.dorecommend    12246
reviews.id             28291
reviews.numhelpful     12217
reviews.rating             0
reviews.sourceurls         0
reviews.text               0
reviews.title              0
reviews.username           5
sourceurls                 0
dtype: int64

In [49]:
df = df.drop(columns=["reviews.didpurchase", "reviews.id", "reviews.numhelpful", "reviews.dorecommend"])

In [50]:
df.isnull().sum()

id                    0
dateadded             0
dateupdated           0
name                  0
asins                 0
brand                 0
categories            0
primarycategories     0
imageurls             0
keys                  0
manufacturer          0
manufacturernumber    0
reviews.date          0
reviews.dateseen      0
reviews.rating        0
reviews.sourceurls    0
reviews.text          0
reviews.title         0
reviews.username      5
sourceurls            0
dtype: int64

# Task 1: Review Classification

## Create sentiment categories

In [10]:
df["reviews.rating"].value_counts()

reviews.rating
5    19897
4     5648
3     1206
1      965
2      616
Name: count, dtype: int64

In [11]:
sentiment_map = {1: "negative", 2: "negative", 3: "neutral", 4: "positive", 5: "positive"}
df["rating_sentiment"] = df["reviews.rating"].map(sentiment_map)

In [12]:
df.head()

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-03-02T00:00:00.000Z,2017-08-28T00:00:00Z,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht...",neutral
1,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-08-31T00:00:00.000Z,2017-08-28T00:00:00Z,4,https://www.amazon.com/product-reviews/B00QWO9...,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...,ByMG,"https://www.barcodable.com/upc/841710106442,ht...",positive
2,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-12-06T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...,BySharon Lambert,"https://www.barcodable.com/upc/841710106442,ht...",positive
3,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2016-03-28T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,Seem to work as well as name brand batteries a...,... as well as name brand batteries at a much ...,Bymark sexson,"https://www.barcodable.com/upc/841710106442,ht...",positive
4,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,HL-002619,2017-04-22T00:00:00.000Z,2017-08-28T00:00:00Z,5,https://www.amazon.com/product-reviews/B00QWO9...,These batteries are very long lasting the pric...,... batteries are very long lasting the price ...,Bylinda,"https://www.barcodable.com/upc/841710106442,ht...",positive


In [13]:
df["rating_sentiment"].value_counts()

rating_sentiment
positive    25545
negative     1581
neutral      1206
Name: count, dtype: int64

In [14]:
percentage_of = round((df["rating_sentiment"].value_counts() / len(df) * 100), 2)

percentage_of

# very imbalanced dataset

rating_sentiment
positive    90.16
negative     5.58
neutral      4.26
Name: count, dtype: float64

## Downsample data for balanced classes

In [15]:
# Split the dataframe into three: positive, negative, neutral
positive = (df[df["rating_sentiment"] == "positive"])
negative = (df[df["rating_sentiment"] == "negative"])
neutral =  (df[df["rating_sentiment"] == "neutral"])

positive_sampled = positive.sample(n=1206)
negative_sampled = negative.sample(n=1206)

len(positive_sampled)

# concat into one balanced dataframe 

df_balanced = pd.concat([negative_sampled, positive_sampled, neutral])

# confirm
df_balanced["rating_sentiment"].value_counts()

rating_sentiment
negative    1206
positive    1206
neutral     1206
Name: count, dtype: int64

## Choose a classifier model

- 0: Negative
- 1: Neutral
- 2: Positive

In [16]:
# pick a model 
# CardiffNLP baseline tokenizes the model
classifier = pipeline(task="sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# view a sample 

sample_reviews = df_balanced["reviews.text"].head(10).tolist()

results = classifier(sample_reviews)
results


Device set to use mps:0


[{'label': 'LABEL_0', 'score': 0.8465775847434998},
 {'label': 'LABEL_2', 'score': 0.8215872645378113},
 {'label': 'LABEL_1', 'score': 0.43460822105407715},
 {'label': 'LABEL_0', 'score': 0.92408287525177},
 {'label': 'LABEL_0', 'score': 0.8691161870956421},
 {'label': 'LABEL_0', 'score': 0.9635699987411499},
 {'label': 'LABEL_0', 'score': 0.6331484317779541},
 {'label': 'LABEL_0', 'score': 0.9678872227668762},
 {'label': 'LABEL_0', 'score': 0.5253010988235474},
 {'label': 'LABEL_0', 'score': 0.6237383484840393}]

In [22]:
sample_sentiment = df_balanced["rating_sentiment"].head(10).tolist
sample_sentiment

<bound method IndexOpsMixin.tolist of 22631    negative
11286    negative
7993     negative
20705    negative
8362     negative
24005    negative
11347    negative
16890    negative
6103     negative
18956    negative
Name: rating_sentiment, dtype: object>

In [23]:
full_results = classifier(df_balanced["reviews.text"].tolist(), truncation=True, max_length=512)

In [24]:
label_strings = [r["label"] for r in full_results]
# returns a list of strings
label_strings
# turn the list of strings into a Series using pd.Series
labels_for_mapping = pd.Series(label_strings)
# now labels are ready for mapping
labels_for_mapping


0       LABEL_0
1       LABEL_0
2       LABEL_0
3       LABEL_0
4       LABEL_0
         ...   
3613    LABEL_0
3614    LABEL_0
3615    LABEL_0
3616    LABEL_0
3617    LABEL_2
Length: 3618, dtype: object

In [25]:
# map the labels into sentiment and create a new column based on the predicted sentiment
label_map = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}
df_balanced["predicted_sentiment"] = labels_for_mapping.map(label_map).values

In [26]:
predicted = df_balanced["predicted_sentiment"]
actual = df_balanced["rating_sentiment"]
print(classification_report(actual, predicted))

              precision    recall  f1-score   support

    negative       0.68      0.76      0.72      1206
     neutral       0.53      0.26      0.35      1206
    positive       0.65      0.91      0.76      1206

    accuracy                           0.64      3618
   macro avg       0.62      0.64      0.61      3618
weighted avg       0.62      0.64      0.61      3618



### Export class-balanced dataset to csv for use in Colab

In [27]:
df_balanced.to_csv('balanced_reviews.csv', index=False)

# 2. Product Category Clustering

In [28]:
df["categories"]

0        AA,AAA,Health,Electronics,Health & Household,C...
1        AA,AAA,Health,Electronics,Health & Household,C...
2        AA,AAA,Health,Electronics,Health & Household,C...
3        AA,AAA,Health,Electronics,Health & Household,C...
4        AA,AAA,Health,Electronics,Health & Household,C...
                               ...                        
28327    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28328    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28329    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28330    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
28331    Fire Tablets,Tablets,All Tablets,Amazon Tablet...
Name: categories, Length: 28332, dtype: object

In [5]:
df["categories"].value_counts()

categories
AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           8343
AA,AAA,Electronics Features,Health,Electro

In [9]:
df["categories"].unique().count()

AttributeError: 'numpy.ndarray' object has no attribute 'count'

## Import local LLM (Qwen2.5) with Ollama for clustering

In [ ]:
# view primary categories as benchmark for meta categories
df["primarycategories"].value_counts()

primarycategories
Electronics                    13995
Health & Beauty                12071
Toys & Games,Electronics        1676
Office Supplies,Electronics      386
Electronics,Media                185
Office Supplies                    9
Animals & Pet Supplies             6
Home & Garden                      2
Electronics,Furniture              2
Name: count, dtype: int64

In [ ]:
# Conduct EDA to understand product categories 

office_supplies = df[df["primarycategories"] == "Office Supplies"]
office_supplies

,id,dateadded,dateupdated,name,asins,brand,categories,primarycategories,imageurls,keys,...,manufacturernumber,reviews.date,reviews.dateseen,reviews.rating,reviews.sourceurls,reviews.text,reviews.title,reviews.username,sourceurls,rating_sentiment
12156,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-03-02T00:00:00.000Z,"2017-09-21T00:00:00Z,2017-08-26T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,This folder also expands on the bottom. Will a...,Bottom expands as well,happypersonohyeh,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12157,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2019-03-25T00:00:00.000Z,"2019-04-22T00:00:00Z,2019-04-19T00:00:00Z",5,https://www.ebay.com/itm/Expanding-Accordion-F...,Exactly what I needed.,Folder,99kurama99,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12158,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-11-09T00:00:00.000Z,2018-06-20T00:00:00Z,5,https://www.ebay.com/itm/Expanding-File-Tabs-F...,Nice product. Will work perfectly for my proje...,Folder,juleekae,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12159,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-03-27T00:00:00.000Z,"2017-10-30T00:00:00Z,2017-09-21T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,A useful folder organizer with good quality.,Good purchase,ml2shop,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12160,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2017-01-12T00:00:00.000Z,"2017-09-21T00:00:00Z,2017-08-26T00:00:00Z,2017...",5,http://www.ebay.com/itm/File-Pocket-Expanding-...,I love the portability of this expandable file...,Great product and cost,connieinla,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12161,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder...",Office Supplies,https://i.ebayimg.com/images/g/qAwAAOSwptJab16...,expandingaccordionfilefolderplasticportabledoc...,...,PBH-2557,2018-06-12T00:00:00.000Z,2018-06-20T00:00:00Z,5,https://www.ebay.com/urw/product-reviews/10014...,Exactly what I needed!!!!,Perfect!,lobo5981,http://www.ebay.com/itm/Expanding-File-Tabs-Fo...,positive
12162,AVpfhcniLJeJML43-t4G,2015-10-30T10:58:06Z,2019-04-22T10:21:26Z,Expanding Accordion File Folder Plastic Portab...,"B0002LCUZK,B010CEC6MI,B01B25NN64",Amazonbasics,"Office,File Jackets & File Pockets,File Folder..

In [21]:
df["primarycategories"]

0        Health & Beauty
1        Health & Beauty
2        Health & Beauty
3        Health & Beauty
4        Health & Beauty
              ...       
28327        Electronics
28328        Electronics
28329        Electronics
28330        Electronics
28331        Electronics
Name: primarycategories, Length: 28332, dtype: object

In [51]:
unique_categories = df["categories"].unique()
unique_categories.tolist()

['AA,AAA,Health,Electronics,Health & Household,Camcorder Batteries,Camera & Photo,Batteries,Household Batteries,Robot Check,Accessories,Camera Batteries,Health and Beauty,Household Supplies,Batteries & Chargers,Health, Household & Baby Care,Health Personal Care',
 'Audio & Video Accessories,TV, Video & Home Audio,Home Audio & Video Accessories,DVD Cases,Electronics,Media Cases & Storage,TV, Video & Audio Accessories,Media Storage & Organization,Disc Storage Wallets,Consumer Electronics,CD, DVD & Blu-ray Discs,Blank Audio/Video Media,Accessories & Supplies,all electronics',
 'Stereos,Remote Controls,Audio Docks & Mini Speakers,Kitchen & Dining Features,Home & Tools,Electronics,New Networking,Wireless Speakers,Home, Garage & Office,Portable Speakers,iPod, Audio Player Accessories,Smart Home,Surveillance,Home Improvement,Smart Home & Home Automation Devices,Alarms & Sensors,Amazon Devices,Amazon,Holiday Shop,Kitchen Dining Features,Amazon Echo,Amazon Echo Accessories,Networking,Speaker Sy

In [52]:
category_map = {}

# use qwen to loop through the categories and create a dictionary of categories and primary categories 

for cat in unique_categories:
    response = ollama.chat(model="qwen2.5", messages=[
        {"role": "user", "content": f"""Classify this product category: '{cat}' into ONE of: 'Electronics', 'Health & Beauty', 'Tablets & E-readers', 'Home & Kitchen', 'Office Supplies', 'Pet Supplies'. Reply with ONLY the category name. 
         DO NOT invent your own category name. Use ONLY the names provided.
         If the product category contains Health & Beauty or Health and Beauty, then the product category is: Health & Beauty
         Litter boxes are classified as Pet Supplies.
         
         """}
    ])
    category_map[cat] = response["message"]["content"]
    print(response["message"]["content"])

Health & Beauty
Electronics
Electronics
Electronics
Electronics
Electronics
Health & Beauty
Electronics
Electronics
Electronics
Electronics
Electronics
Office Supplies
Pet Supplies
Electronics
Electronics
Home & Kitchen
Electronics
Electronics
Electronics
Tablets & E-readers
Electronics
Pet Supplies
Electronics
Electronics
Electronics
Tablets & E-readers
Home & Kitchen
Electronics
Tablets & E-readers
Electronics
Electronics
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Pet Supplies
Tablets & E-readers
Pet Supplies
Electronics
Electronics
Tablets & E-readers
Electronics
Electronics
Tablets & E-readers
Tablets & E-readers
Tablets & E-readers
Electronics
Tablets & E-readers
Electronics
Electronics
Electronics
Electronics
Electronics


In [24]:
# turn the dictionary into a dataframe 
# use .items() to split the key:value pairs into a list of tuples so pd.Dataframe can create two colums, key and value

mapping_df = pd.DataFrame(category_map.items(), columns=["categories", "primary_category"])
mapping_df.head()

,categories,primary_category
0,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty
1,"Audio & Video Accessories,TV, Video & Home Aud...",Electronics
2,"Stereos,Remote Controls,Audio Docks & Mini Spe...",Electronics
3,"Amazon Echo,Virtual Assistant Speakers,Electro...",Electronics
4,"Amazon Echo,Networking,Home & Tools,Smart Home...",Electronics


In [ ]:
df["meta_category"] = df["categories"].map(category_map) # map the LLM's primary categories to the dataframe's categories as "meta_category"

In [26]:

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
df[["categories", "primarycategories", "meta_category"]].value_counts()

categories                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  primarycategories            meta_category      
AA,AAA,Heal

In [27]:
df.columns

Index(['id', 'dateadded', 'dateupdated', 'name', 'asins', 'brand',
       'categories', 'primarycategories', 'imageurls', 'keys', 'manufacturer',
       'manufacturernumber', 'reviews.date', 'reviews.dateseen',
       'reviews.rating', 'reviews.sourceurls', 'reviews.text', 'reviews.title',
       'reviews.username', 'sourceurls', 'rating_sentiment', 'meta_category'],
      dtype='object')

## Save the df with meta_categories and rating_sentiment labels as a csv to save tokens & time when reloading

In [28]:
df.to_csv("amazon_sentiment_categories.csv", index=False)

In [5]:
new_df = pd.read_csv("/Users/chandlershortlidge/Desktop/Ironhack/project-dsai-business-case-automated-customer-reviews/notebooks/amazon_sentiment_categories.csv")

# Task 3: Review Summarization Using Generative AI

In [3]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
new_df.groupby(['meta_category', 'name'])['reviews.rating'].mean().sort_values()

meta_category        name                                                                                                                                                  
Tablets & E-readers  Oem Amazon Kindle Power Usb Adapter Wall Travel Charger Fire/dx/+micro Usb Cable                                                                          1.000000
Electronics          Certified Refurbished Amazon Fire TV with Alexa Voice Remote                                                                                              2.800000
                     Amazon Kindle Replacement Power Adapter (Fits Latest Generation Kindle and Kindle DX) For shipment in the U.S only                                        2.800000
Pet Supplies         AmazonBasics Double-Door Folding Metal Dog Crate - Medium (36x23x25 Inches)                                                                               3.500000
Home & Kitchen       AmazonBasics Silicone Hot Handle Cover/Holder - Red                    

In [8]:
def generate_category_blog(df, category, summaries, model='qwen2.5', n_top=3):
    """
    For a given meta_category:
      - Finds the top N products by review count
      - Finds the worst product by average rating
      - Generates a blog-style summary via Ollama
      - Stores the result in summaries[category_key] and prints it
    """
    # Filter to category
    cat_df = df[df["meta_category"] == category]

    # Top N products by number of reviews
    top_n = (
        cat_df.groupby(['meta_category', 'name'])
        .agg(avg_rating=('reviews.rating', 'mean'), num_reviews=('reviews.rating', 'count'))
        .sort_values(by='num_reviews', ascending=False)
        .head(n_top)
    )

    # Reviews for each top product
    top_names = top_n.index.get_level_values('name').tolist()
    top_reviews = [df[df["name"] == name]["reviews.text"].tolist()[:30] for name in top_names]

    # Worst product by average rating
    worst_product = (
        cat_df.groupby(['meta_category', 'name'])
        .agg(avg_rating=('reviews.rating', 'mean'), num_reviews=('reviews.rating', 'count'))
        .sort_values(by='avg_rating', ascending=True)
        .head(1)
    )
    worst_name = worst_product.index.get_level_values('name')[0]
    worst_reviews = df[df["name"] == worst_name]["reviews.text"].tolist()

    # Build a readable review block for the prompt
    top_reviews_str = "\n\n".join(
        [f"Product {i+1} - {top_names[i]}:\n{top_reviews[i]}" for i in range(len(top_names))]
    )

    # Call Ollama
    response = ollama.chat(model=model, messages=[
        {"role": "user", "content": f"""
Write a short article (like a blog post) about the product category: {category}. The output should include:

- Top {n_top} products {top_n} and key differences between them.
- Reviews and top complaints for each: {top_reviews_str}

-----
You should also include the worst product {worst_product} in the category and why it should be avoided.
Worst product reviews: {worst_reviews}
Do not forget to include this.
----

Make sure the style is like a blog.
"""}
    ])

    # Normalise category name to a dict key (e.g. "Health & Beauty" -> "health_beauty")
    category_key = category.lower().replace(" & ", "_").replace("&", "_").replace(" ", "_")
    summaries[category_key] = response['message']['content']
    print(response['message']['content'])
    return response['message']['content']

In [9]:
# Generate blog summaries for all categories in one go
categories = [
    "Electronics",
    "Tablets & E-readers",
    "Health & Beauty",
    "Home & Kitchen",
    "Office Supplies",
    "Pet Supplies",
]

summaries = {}
for cat in categories:
    print(f"\n{'='*60}\n{cat}\n{'='*60}")
    generate_category_blog(new_df, cat, summaries)

with open("summaries.json", "w") as f:
    json.dump(summaries, f)


Electronics
### The Worst Product in Our Category: A Cautionary Tale

In an era where technology upgrades happen at lightning speed, it’s crucial to choose reliable and durable products. However, sometimes even tech giants can release subpar items that leave customers disappointed. Today, we’re diving into the worst product within our electronics category—specifically focusing on a power adapter for Amazon Kindle devices. This adapter, which should be providing essential charging support, has become more of an inconvenience than a solution.

#### The Product in Question
The infamous "Amazon Kindle Replacement Power Adapter (Fits L...)" is a prime example of what can go wrong with even the most popular tech brands. Let’s take a closer look at why this adapter deserves to be avoided:

1. **Premature Deterioration:**
   - One user reported that after only one use, the outer covering of the cord started cracking and falling off in bits. This is incredibly frustrating, as it renders the ch